In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

import torch

from torch.nn import Conv1d

from torch.nn import MaxPool1d

from torch.nn import Flatten

from torch.nn import Linear

from torch.nn.functional import relu

from torch.utils.data import DataLoader, TensorDataset

from torch.optim import SGD

from torch.nn import L1Loss

!pip install pytorch-ignite

from ignite.contrib.metrics.regression.r2_score import R2Score

     |████████████████████████████████| 112kB 3.4MB/s 


In [0]:
class CnnRegressor(torch.nn.Module):

  def __init__(self, batch_size, inputs, outputs, kernelsize):

    super(CnnRegressor, self).__init__()
    self.batch_size = batch_size
    self.inputs = inputs
    self.outputs = outputs

    self.input_layer = Conv1d(inputs, batch_size, kernelsize)

    self.max_pooling_layer = MaxPool1d(1)

    self.conv_layer = Conv1d(batch_size, 128, kernelsize)

    self.flatten_layer = Flatten()

    self.linear_layer = Linear(128, 64)

    self.output_layer = Linear(64, outputs)

  
  def feed(self, input):

    input = input.reshape((self.batch_size, self.inputs, 1))

    output = relu(self.input_layer(input))

    output = self.max_pooling_layer(output)

    output = relu(self.conv_layer(output))

    output = self.flatten_layer(output)

    output = self.linear_layer(output)

    output = self.output_layer(output)

    return output

def model_loss(model, dataset, train = False, optimizer = 'adam'):

  performance = L1Loss()
  score_metric = R2Score()

  avg_loss = 0
  avg_score = 0
  count = 0

  for input, output in iter(dataset):

    predictions = model.feed(input)

    loss = performance(predictions, output)

    score_metric.update([predictions, output])
    score = score_metric.compute()
        
    if(train):

      optimizer.zero_grad()

      loss.backward()

      optimizer.step()

    avg_loss += loss.item()
    avg_score += score
    count += 1

  return avg_loss / count, avg_score / count

In [0]:
dataset = pd.read_csv('housing.csv')

dataset = dataset.dropna()

In [0]:
Y = dataset['median_house_value']
X = dataset.loc[:,'longitude':'median_income']

In [0]:
# 30% for testing dataset
__testPercent__ = 0.3
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=__testPercent__)

x_train_np = x_train.to_numpy()

y_train_np = y_train.to_numpy()

x_test_np = x_test.to_numpy()
y_test_np = y_test.to_numpy()

In [6]:
ResultArry = {}

for x in range(1,10):

  # To see and compare the final results
  currentID = "kernel-"+str(x)

  batch_size = 38

  model = CnnRegressor(batch_size, X.shape[1], 1,x)

  model.cuda()

  epochs = 50

  optimizer = SGD(model.parameters(), lr=1e-5)
  reshapeddata = y_train_np.reshape(y_train_np.shape[0],1)
  inputs = torch.from_numpy(x_train_np).cuda().float()
  outputs = torch.from_numpy(y_train_np.reshape(y_train_np.shape[0],1)).cuda().float()

  tensor = TensorDataset(inputs, outputs)

  loader = DataLoader(tensor, batch_size, shuffle=True, drop_last=True)

  for epoch in range(epochs):
    avg_loss, avg_r2_score = model_loss(model, loader, train=True, optimizer=optimizer)
    # print("Epoch "+ str(epoch + 1) + ":\n\tLoss = " + str(avg_loss) + "\n\r R2 score = " + str(avg_r2_score)  )

  inputs = torch.from_numpy(x_test_np).cuda().float()
  outputs = torch.from_numpy(y_test_np.reshape(y_test_np.shape[0],1)).cuda().float()

  tensor = TensorDataset(inputs, outputs)
  loader = DataLoader(tensor, batch_size, shuffle=True, drop_last=True)

  avg_loss, avg_r2_score = model_loss(model, loader)
  # print("the model L1 loss is: " + str(avg_loss))
  # print("the model R2 score is: " + str(avg_r2_score))

  ResultArry.update({currentID:avg_r2_score})

  print("now:"+str(currentID)+' ++')
  print(ResultArry)

print(ResultArry)
#get max accuracy
maxkey=0
maxval = -9999999
for key in ResultArry:
  if(ResultArry[key]>maxval):
      maxval = ResultArry[key]
      maxkey = key

print('best result was for:')
print(str(maxkey) + ' -> ' + str(ResultArry[maxkey]))

now:kernel-1 ++
{'kernel-1': 0.3014435231076445}


RuntimeError: ignored